<a href="https://colab.research.google.com/github/a-manic/Art-generation-using-GANS/blob/main/Art_Generation_using_GANs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from tqdm.notebook import tqdm
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.utils import save_image
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [ ]:
batch_size = 128
image_size = (64,64) #all images are resized to this size 
device = get_default_device()
latent_size = 150 #can be changed
fixed_latent = torch.randn(64, latent_size, 1, 1, device=device)
lr = 0.0015
epochs = 300


input_dir = "/content/gdrive/MyDrive/ECE 283 Project/1/"
output_dir = "/content/gdrive/MyDrive/ECE 283 Project/1_lr0.0015_generated/"
os.makedirs(output_dir, exist_ok = True)
device

device(type='cuda')

In [ ]:
transform_dataset = transforms.Compose([transforms.Resize(image_size), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_dataset = torchvision.datasets.ImageFolder(root=input_dir, transform=transform_dataset)

train_data = DataLoader(train_dataset, batch_size, shuffle=True, num_workers=3, pin_memory=True)

train_data = DeviceDataLoader(train_data, device)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
def denormalize(img_tensors):
    #denormalizes the image noramlized in the transforms
    return img_tensors * 0.5 + 0.5


In [ ]:
discriminator = nn.Sequential(
    # in: 3 x 64 x 64

    nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=0, bias=False),
    nn.BatchNorm2d(64),
    nn.LeakyReLU(0.2, inplace=True),
    # out: 64 x 31 x 31

    nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=0, bias=False),
    nn.BatchNorm2d(128),
    nn.LeakyReLU(0.2, inplace=True),
    # out: 128 x 14 x 14

    nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=0, bias=False),
    nn.BatchNorm2d(256),
    nn.LeakyReLU(0.2, inplace=True),
    # out: 256 x 6 x 6

    nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=0, bias=False),
    nn.BatchNorm2d(512),
    nn.LeakyReLU(0.2, inplace=True),
    # out: 512 x 2 x 2

    nn.Conv2d(512, 1, kernel_size=2, stride=1, padding=0, bias=False),
    # out: 1 x 1 x 1

    nn.Flatten(),
    # out : 1
    nn.Sigmoid()
    # out : 1
    )

In [ ]:
generator = nn.Sequential(
    # in: latent_size x 1 x 1

    nn.ConvTranspose2d(latent_size, 512, kernel_size=4, stride=1, padding=0, bias=False),
    nn.BatchNorm2d(512),
    nn.ReLU(True),
    # out: 512 x 4 x 4

    nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(256),
    nn.ReLU(True),
    # out: 256 x 8 x 8

    nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(128),
    nn.ReLU(True),
    # out: 128 x 16 x 16

    nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(64),
    nn.ReLU(True),
    # out: 64 x 32 x 32

    nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1, bias=False),
    nn.Tanh()
    # out: 3 x 64 x 64
)

In [ ]:
discriminator = to_device(discriminator, device)
generator = to_device(generator, device)

In [ ]:
def train_discriminator(real_images, optimizer_dis):
    # Clear discriminator gradients
    optimizer_dis.zero_grad()

    # Pass real images through discriminator
    real_predicted_labels = discriminator(real_images)
    real_true_lables = torch.ones(real_images.size(0), 1, device=device)
    real_data_loss = F.binary_cross_entropy(real_predicted_labels, real_true_lables)
    real_accuracy = torch.mean(real_predicted_labels).item()
    
    # Generate fake images
    latent = torch.randn(batch_size, latent_size, 1, 1, device=device)
    generated_images = generator(latent)

    # Pass generated images through discriminator
    fake_predicted_labels = discriminator(generated_images)
    fake_true_labels = torch.zeros(generated_images.size(0), 1, device=device)
    fake_data_loss = F.binary_cross_entropy(fake_predicted_labels, fake_true_labels)
    fake_accuracy = torch.mean(fake_predicted_labels).item()

    # Update discriminator weights and compute total discriminator loss
    discriminator_loss = real_data_loss + fake_data_loss
    discriminator_loss.backward()
    optimizer_dis.step()
    return discriminator_loss.item(), real_accuracy, fake_accuracy

In [ ]:
def train_generator(optimizer_gen):
    # Clear generator gradients
    optimizer_gen.zero_grad()
    
    # Generate fake images
    latent = torch.randn(batch_size, latent_size, 1, 1, device=device)
    fake_images = generator(latent)
    
    # Pass generated images through discriminator
    fake_predicted_labels = discriminator(fake_images)
    fake_true_labels = torch.ones(batch_size, 1, device=device)
    generator_loss = F.binary_cross_entropy(fake_predicted_labels, fake_true_labels)
    
    # Update generator weights and compute generator loss
    generator_loss.backward()
    optimizer_gen.step()
    
    return generator_loss.item()

In [ ]:
def save_samples(index, latent_tensors):
    fake_images = generator(latent_tensors)
    img_name = 'generated-images-{0:0=4d}.png'.format(index)
    save_image(denormalize(fake_images), os.path.join(output_dir, img_name), nrow=8)
    print('Saving', img_name)

In [ ]:
def fit(epochs, lr, start_index=1):
    torch.cuda.empty_cache()
    
    # Losses & scores
    loss_discriminator = []
    loss_generator = []
    real_accuracy_prob = []
    fake_accuracy_prob = []
    
    # Create optimizers
    optimizer_dis = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))
    optimizer_gen = torch.optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
    
    for epoch in range(epochs):
        for real_images, _ in tqdm(train_data):
            # Train discriminator
            discriminator_loss, real_accuracy, fake_accuracy = train_discriminator(real_images, optimizer_dis)
            # Train generator
            generator_loss = train_generator(optimizer_gen)
            
        # Record losses & scores
        loss_discriminator.append(discriminator_loss)
        loss_generator.append(generator_loss)
        real_accuracy_prob.append(real_accuracy)
        fake_accuracy_prob.append(fake_score)
        
        # Log losses & scores (last batch)
        print("Epoch [{}/{}], discriminator_loss: {:.4f}, generator_loss: {:.4f}, real_accuracy: {:.4f}, fake_accuracy: {:.4f}".format(
            epoch+1, epochs, discriminator_loss, generator_loss, real_accuracy, fake_accuracy))
    
        # save_samples(epoch+start_index, fixed_latent)
    
    return loss_discriminator, loss_generator, real_accuracy_prob, fake_accuracy_prob

In [ ]:
history = fit(epochs,lr)

In [ ]:
loss_discriminator, loss_generator, real_accuracy_prob, fake_accuracy_prob = history

In [ ]:
torch.save(generator.state_dict(), 'Generator.ckpt')
torch.save(discriminator.state_dict(), 'Discriminator.ckpt')

In [ ]:
plt.plot(loss_discriminator, '-')
plt.plot(loss_generator, '-')
plt.xlabel('Epochs')
plt.ylabel('Cross Entropy Loss')
plt.legend(['Discriminator', 'Generator'])
plt.title('Cross-Entropy Losses of Discriminator and Generator');

In [ ]:
plt.plot(real_accuracy_prob, '-')
plt.plot(fake_accuracy_prob, '-')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Real', 'Fake'])
plt.title('Accuracy of the model');

In [ ]:
vid_fname = '{}/gans_training.avi'.format(output_dir)

files = [os.path.join(output_dir, f) for f in os.listdir(output_dir) if 'generated' in f]
files.sort()

out = cv2.VideoWriter(vid_fname,cv2.VideoWriter_fourcc(*'MP4V'), 1, (530,530))
[out.write(cv2.imread(fname)) for fname in files]
out.release()